In [4]:
!az login

ERROR: User cancelled the Accounts Control Operation.. Status: Response_Status.Status_UserCanceled, Error code: 0, Tag: 528315210
Run the command below to authenticate interactively; additional arguments may be added as needed:
az logout
az login


In [ ]:
!pip uninstall azure-ai-projects
!pip install openai azure-identity python-dotenv

^C


In [ ]:
# pylint: disable=line-too-long,useless-suppression
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to run Prompt Agent operations
    using MCP (Model Context Protocol) tools and a synchronous client.

USAGE:
    python sample_agent_mcp.py

    Before running the sample:

    pip install "azure-ai-projects>=2.0.0b1" azure-identity openai python-dotenv

    Set these environment variables with your own values:
    1) AZURE_AI_PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
       page of your Microsoft Foundry portal.
    2) AZURE_AI_MODEL_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in
       the "Models + endpoints" tab in your Microsoft Foundry project.
"""

import os
from dotenv import load_dotenv
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam



load_dotenv()

endpoint = os.environ["PROJECT_ENDPOINT"]

with (
    AzureCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID")) as credential,
    AIProjectClient(endpoint=endpoint, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):
    
  
    # [START tool_declaration]
    mcp_tool = MCPTool(
        server_label="custom-web-search",
        server_url="http://20.81.137.101:9000/mcp",
        allowed_tools=["duckduckgo_search"],
        require_approval="never",
        project_connection_id="custom-web-search"
    )
    # [END tool_declaration]

    agent = project_client.agents.update(
        agent_name="new-web-search-agent",
        definition=PromptAgentDefinition(
            model="gpt-5-nano",
            instructions='''You are an helpful web-search agent. 
                            Please find the required answer from web search utilizing the tool. 
                            Do not make any answer on your own. Respond in following format.
                            info from Web Search:
                            <Summarize the information retrieved for the user query from web, 
                            if no relevant content found then respond "No information found in Web">''',
            tools=[mcp_tool],
            
        ),
    )
    print(f"Agent created (id: {agent.id}, name: {agent.name})")

    # # Create a conversation thread to maintain context across multiple interactions
    # conversation = openai_client.conversations.create()
    # print(f"Created conversation (id: {conversation.id})")

    # # Send initial request that will trigger the MCP tool
    # response = openai_client.responses.create(
    #     conversation=conversation.id,
    #     input="Please summarize the Azure REST API specifications Readme",
    #     extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    # )

    # # Process any MCP approval requests that were generated
    # input_list: ResponseInputParam = []
    # for item in response.output:
    #     if item.type == "mcp_approval_request":
    #         if item.server_label == "api-specs" and item.id:
    #             # Automatically approve the MCP request to allow the agent to proceed
    #             # In production, you might want to implement more sophisticated approval logic
    #             input_list.append(
    #                 McpApprovalResponse(
    #                     type="mcp_approval_response",
    #                     approve=True,
    #                     approval_request_id=item.id,
    #                 )
    #             )

    # print("Final input:")
    # print(input_list)

    # # Send the approval response back to continue the agent's work
    # # This allows the MCP tool to access the GitHub repository and complete the original request
    # response = openai_client.responses.create(
    #     input=input_list,
    #     previous_response_id=response.id,
    #     extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    # )

    # print(f"Response: {response.output_text}")

    # # Clean up resources by deleting the agent version
    # # This prevents accumulation of unused agent versions in your project
    # project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
    # print("Agent deleted")

Agent created (id: new-web-search-agent, name: new-web-search-agent)
